Connecting ADLS to Databricks

In [0]:
dbutils.fs.mount(
    source = "wasbs://retail@sgretailproject.blob.core.windows.net",
    mount_point = "/mnt/retail",
    extra_configs = {"fs.azure.account.key.sgretailproject.blob.core.windows.net": "CCWg3BZm5iJ5Bs8kM1ZrLqMUZrI4HiwVtD3otKrSvQUFe+3ibx/w+RSndmRdtjAkwMXQEw+7XZms+AStiE3W3Q=="}
)

True

In [0]:
dbutils.fs.ls("/mnt/retail/bronze/transaction/")

[FileInfo(path='dbfs:/mnt/retail/bronze/transaction/dbo.transactions.parquet', name='dbo.transactions.parquet', size=16222, modificationTime=1753264702000)]

Reading the bronze layer

In [0]:
df_products = spark.read.parquet("/mnt/retail/bronze/products/")
display(df_products)

product_id,product_name,category,price
1,Wireless Mouse,Electronics,799.99
2,Bluetooth Speaker,Electronics,1299.49
3,Yoga Mat,Fitness,499.00
4,Laptop Stand,Accessories,999.99
5,Notebook Set,Stationery,149.00
6,Water Bottle,Fitness,299.00
7,Smartwatch,Electronics,4999.00
8,Desk Organizer,Accessories,399.00
9,Dumbbell Set,Fitness,1999.00
10,Pen Drive 32GB,Electronics,599.00


In [0]:
df_transactions = spark.read.parquet("/mnt/retail/bronze/transaction/")
df_store = spark.read.parquet("/mnt/retail/bronze/store/")
df_customer = spark.read.parquet("/mnt/retail/bronze/customer/unitinguncle/ADF-Retail-Project/refs/heads/main")

Creating silver layer - Data Cleaning

In [0]:
from pyspark.sql.functions import col

#converting types and cleaning data
df_transactions = df_transactions.select(
    col("transaction_id").cast("integer"),
    col("customer_id").cast("integer"),
    col("product_id").cast("integer"),
    col("store_id").cast("integer"),
    col("quantity").cast("integer"),
    col("transaction_date").cast("date")
)

In [0]:
df_products = df_products.select(
    col("product_id").cast("integer"),
    col("product_name"),
    col("category"),
    col("price").cast("decimal(10,2)")
)
df_store = df_store.select(
    col("store_id").cast("integer"),
    col("store_name"),
    col("location")
)
df_customer = df_customer.select(
    col("customer_id").cast("integer"),
    col("first_name"),
    col("last_name"),
    col("email"),
    col("phone"),
    col("city"),
    col("registration_date").cast("date")
)

Joining all data together to create silver layer

In [0]:
df_silver = df_transactions.join(df_customer, "customer_id").join(df_products, "product_id").join(df_store, "store_id").withColumn("Total_amount",col("quantity")*col("price"))


In [0]:
display(df_silver)

store_id,product_id,customer_id,transaction_id,quantity,transaction_date,first_name,last_name,email,phone,city,registration_date,product_name,category,price,store_name,location,Total_amount
6,1,194,1,3,2024-07-01,Meena,Gupta,user194@example.com,9894479769,Delhi,2024-02-11,Wireless Mouse,Electronics,799.99,Urban Central,Hyderabad,2399.97
12,7,152,2,3,2024-05-22,Rahul,Sharma,user152@example.com,9879732377,Ahmedabad,2022-12-09,Smartwatch,Electronics,4999.00,Stationery Spot,Nagpur,14997.00
16,99,184,3,2,2024-12-09,Ravi,Chopra,user184@example.com,9894548726,Jaipur,2024-04-30,Foam Mat,Fitness,799.00,The Fit Store,Indore,1598.00
7,55,186,4,5,2025-10-25,Suresh,Verma,user186@example.com,9864493088,Delhi,2022-07-27,Stretch Strap,Fitness,279.00,Electro Hub,Ahmedabad,1395.00
17,42,105,5,1,2024-05-27,Aman,Verma,user105@example.com,9841425414,Jaipur,2022-06-29,Fitness Scale,Electronics,1999.00,Mega Mart Express,Vadodara,1999.00
5,82,181,6,4,2024-08-27,Ravi,Chopra,user181@example.com,9843594829,Lucknow,2023-12-11,Digital Photo Frame,Electronics,2499.00,Mega Plaza,Chennai,9996.00
4,38,169,7,3,2025-10-30,Rahul,Chopra,user169@example.com,9896866745,Delhi,2024-05-30,Smart Thermometer,Electronics,1099.00,Downtown Mini Store,Pune,3297.00
2,59,198,8,5,2024-02-21,Deepa,Reddy,user198@example.com,9867072320,Chennai,2022-07-27,Aerobic Stepper,Fitness,1499.00,High Street Store,Delhi,7495.00
15,96,165,9,1,2025-12-15,Anjali,Verma,user165@example.com,9849356407,Jaipur,2022-04-07,Document Holder,Accessories,499.00,Mini Market Outlet,Thiruvananthapuram,499.00
10,75,104,10,4,2025-02-05,Ravi,Singh,user104@example.com,9815057178,Hyderabad,2022-05-14,Hand Grip,Fitness,299.00,Smart Buy Store,Surat,1196.00


Dumping data in to silver layer

In [0]:
silver_path = "/mnt/retail/silver/"
df_silver.write.mode("overwrite").format("delta").save(silver_path)

Creating silver data set

In [0]:
spark.sql("DROP TABLE IF EXISTS silver")
spark.sql(f"CREATE TABLE silver USING DELTA LOCATION '{silver_path}'")

DataFrame[]

In [0]:
%sql SELECT * FROM silver LIMIT 3;

store_id,product_id,customer_id,transaction_id,quantity,transaction_date,first_name,last_name,email,phone,city,registration_date,product_name,category,price,store_name,location,Total_amount
6,1,194,1,3,2024-07-01,Meena,Gupta,user194@example.com,9894479769,Delhi,2024-02-11,Wireless Mouse,Electronics,799.99,Urban Central,Hyderabad,2399.97
12,7,152,2,3,2024-05-22,Rahul,Sharma,user152@example.com,9879732377,Ahmedabad,2022-12-09,Smartwatch,Electronics,4999.00,Stationery Spot,Nagpur,14997.00
16,99,184,3,2,2024-12-09,Ravi,Chopra,user184@example.com,9894548726,Jaipur,2024-04-30,Foam Mat,Fitness,799.00,The Fit Store,Indore,1598.00


Creating gold layer

In [0]:
silver_df = spark.read.format("delta").load(silver_path)

In [0]:
from pyspark.sql.functions import sum, countDistinct, avg

gold_df = silver_df.groupBy(
    "transaction_date", "product_id", "product_name", "category", "store_id", "store_name", "location"
).agg(
    sum("quantity").alias("Total_quantity_sold"),
    sum("total_amount").alias("Total_sales_amount"),
    countDistinct("customer_id").alias("Total_customers"),
    countDistinct("transaction_id").alias("number_of_transactions"),
    avg("price").alias("Average_price"),
    avg("total_amount").alias("Average_transaction_value")
)

In [0]:
display(gold_df)

transaction_date,product_id,product_name,category,store_id,store_name,location,Total_quantity_sold,Total_sales_amount,Total_customers,number_of_transactions,Average_price,Average_transaction_value
2024-05-22,7,Smartwatch,Electronics,12,Stationery Spot,Nagpur,3,14997.00,1,1,4999.000000,14997.000000
2025-04-07,79,Weighted Vest,Fitness,14,Tech Arena,Bhopal,4,7996.00,1,1,1999.000000,7996.000000
2024-03-12,56,Travel Adapter,Accessories,12,Stationery Spot,Nagpur,4,1596.00,1,1,399.000000,1596.000000
2025-08-26,32,Tech Organizer Case,Accessories,1,City Mall Store,Mumbai,1,549.00,1,1,549.000000,549.000000
2024-04-27,52,Monitor Stand,Accessories,3,Tech World Outlet,Bangalore,4,5996.00,1,1,1499.000000,5996.000000
2025-08-28,95,Yoga Towel,Fitness,7,Electro Hub,Ahmedabad,1,499.00,1,1,499.000000,499.000000
2025-09-01,50,Laptop Cooling Pad,Electronics,13,Gadget Lane,Patna,4,3996.00,1,1,999.000000,3996.000000
2025-03-04,66,Smart Plug,Electronics,12,Stationery Spot,Nagpur,2,1498.00,1,1,749.000000,1498.000000
2024-03-13,29,Whiteboard Markers,Stationery,13,Gadget Lane,Patna,3,387.00,1,1,129.000000,387.000000
2024-12-01,46,Streaming Stick,Electronics,14,Tech Arena,Bhopal,4,13996.00,1,1,3499.000000,13996.000000


In [0]:
gold_path = "/mnt/retail/gold/"
gold_df.write.mode("overwrite").format("delta").save(gold_path)
spark.sql("DROP TABLE IF EXISTS gold")
spark.sql(f"CREATE TABLE gold USING DELTA LOCATION '{gold_path}'")

DataFrame[]

In [0]:
%sql select * from gold

transaction_date,product_id,product_name,category,store_id,store_name,location,Total_quantity_sold,Total_sales_amount,Total_customers,number_of_transactions,Average_price,Average_transaction_value
2024-05-22,7,Smartwatch,Electronics,12,Stationery Spot,Nagpur,3,14997.00,1,1,4999.000000,14997.000000
2025-04-07,79,Weighted Vest,Fitness,14,Tech Arena,Bhopal,4,7996.00,1,1,1999.000000,7996.000000
2024-03-12,56,Travel Adapter,Accessories,12,Stationery Spot,Nagpur,4,1596.00,1,1,399.000000,1596.000000
2025-08-26,32,Tech Organizer Case,Accessories,1,City Mall Store,Mumbai,1,549.00,1,1,549.000000,549.000000
2024-04-27,52,Monitor Stand,Accessories,3,Tech World Outlet,Bangalore,4,5996.00,1,1,1499.000000,5996.000000
2025-08-28,95,Yoga Towel,Fitness,7,Electro Hub,Ahmedabad,1,499.00,1,1,499.000000,499.000000
2025-09-01,50,Laptop Cooling Pad,Electronics,13,Gadget Lane,Patna,4,3996.00,1,1,999.000000,3996.000000
2025-03-04,66,Smart Plug,Electronics,12,Stationery Spot,Nagpur,2,1498.00,1,1,749.000000,1498.000000
2024-03-13,29,Whiteboard Markers,Stationery,13,Gadget Lane,Patna,3,387.00,1,1,129.000000,387.000000
2024-12-01,46,Streaming Stick,Electronics,14,Tech Arena,Bhopal,4,13996.00,1,1,3499.000000,13996.000000
